# Test Model Training

In [ ]:
import sys

In [ ]:
sys.path.insert(0,'/run/determined/workdir/demo_revamp/determined/examples/computer_vision/detectron2_coco_pytorch/')

In [ ]:
sys.path

In [ ]:
import detectron2_files

In [ ]:
import torch
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data.catalog import DatasetCatalog, MetadataCatalog

from detectron2.modeling import build_model
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.utils.events import EventStorage
from detectron2_files.common import *
from detectron2_files.data import *
from detectron2_files.evaluator import *
from torch import nn
from pprint import pprint
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.data.datasets import register_coco_instances
import random
from detectron2.utils.visualizer import Visualizer
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
cfg = get_cfg()

In [ ]:
cfg.DATASETS.TRAIN

In [ ]:
# YAML_PATH = '/run/determined/workdir/demo_revamp/determined/examples/computer_vision/detectron2_coco_pytorch/models/mask_rcnn_R_50_FPN_noaug_1x.yaml'
YAML_PATH = '/run/determined/workdir/demo_revamp/determined/examples/computer_vision/detectron2_coco_pytorch/models/fast_rcnn_R_50_FPN_1x.yaml'

cfg.merge_from_file(YAML_PATH)

In [ ]:
# print(cfg.dump())

In [ ]:
# import json

In [ ]:
# train_d = json.load(open('/run/determined/workdir/demo_revamp/determined/tiny-coco/small_coco/instances_train2017_small.json','r'))

In [ ]:
# DatasetCatalog.register("test_dataset",lambda: train_d)

In [ ]:
# MetadataCatalog.get("test_dataset")

In [ ]:
roboflow_dataset_dirs = {
        'x-ray-rheumatology': '/run/determined/workdir/shared_fs/data/x-ray-rheumatology',
        'flir-camera-objects': '/run/determined/workdir/shared_fs/data/flir-camera-objects',
    }

def register_roboflow_dataset(dataset_name,cfg):
    '''
    '''
    cfg.defrost()
    DIR = roboflow_dataset_dirs[dataset_name]
    print("DIR: ",DIR)
    TRAIN_JSON_PATH = os.path.join(DIR,'train/_annotations.coco.json')
    TRAIN_IMG_DIR = os.path.join(DIR,'train/')
    VAL_JSON_PATH = os.path.join(DIR,'valid/_annotations.coco.json')
    VAL_IMG_DIR = os.path.join(DIR,'valid/')
    TEST_JSON_PATH = os.path.join(DIR,'test/_annotations.coco.json')
    TEST_IMG_DIR = os.path.join(DIR,'test/')
    train_dataset_name = "train_{}".format(dataset_name)
    val_dataset_name="val_{}".format(dataset_name)
    test_dataset_name = "test_{}".format(dataset_name)
    register_coco_instances(train_dataset_name, {}, TRAIN_JSON_PATH, TRAIN_IMG_DIR)
    register_coco_instances(val_dataset_name, {}, VAL_JSON_PATH, VAL_IMG_DIR)
    register_coco_instances(test_dataset_name, {}, TEST_JSON_PATH, TEST_IMG_DIR)
    cfg.DATASETS.TRAIN=("train_{}".format(dataset_name),)
    cfg.DATASETS.VAL=("val_{}".format(dataset_name),)
    cfg.DATASETS.TEST=("test_{}".format(dataset_name),)
    # cfg.freeze()
    return cfg, train_dataset_name, val_dataset_name, test_dataset_name

In [ ]:
dataset_name = 'flir-camera-objects'

In [ ]:
# register_coco_instances("coco_custom", {}, "/run/determined/workdir/demo_revamp/determined/tiny-coco/small_coco/instances_train2017_small.json", "/run/determined/workdir/demo_revamp/determined/tiny-coco/small_coco/train_2017_small")
# DIR = '/run/determined/workdir/shared_fs/data/'
# TRAIN_JSON_PATH = os.path.join(DIR,'x-ray-rheumatology-2/train/_annotations.coco.json')
# TRAIN_IMG_DIR = os.path.join(DIR,'x-ray-rheumatology-2/train/')
# VAL_JSON_PATH = os.path.join(DIR,'x-ray-rheumatology-2/val/_annotations.coco.json')
# VAL_IMG_DIR = os.path.join(DIR,'x-ray-rheumatology-2/val/')
# TEST_JSON_PATH = os.path.join(DIR,'x-ray-rheumatology-2/test/_annotations.coco.json')
# TEST_IMG_DIR = os.path.join(DIR,'x-ray-rheumatology-2/test/')
# TRAIN_JSON_PATH

cfg, train_dataset_name, val_dataset_name, test_dataset_name = register_roboflow_dataset(dataset_name,cfg)

In [ ]:
# register_coco_instances("train_med_coco", {}, TRAIN_JSON_PATH, TRAIN_IMG_DIR)
# register_coco_instances("val_med_coco", {}, VAL_JSON_PATH, VAL_IMG_DIR)
# register_coco_instances("test_med_coco", {}, TEST_JSON_PATH, TEST_IMG_DIR)

In [ ]:

nuts_metadata = MetadataCatalog.get(train_dataset_name)
dataset_dicts = DatasetCatalog.get(train_dataset_name)

In [ ]:
nuts_metadata.thing_classes

In [ ]:

for d in random.sample(dataset_dicts, 3):
    print(d)
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :], metadata=nuts_metadata , scale=1.2)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :])
    plt.show()

In [ ]:
# cfg.DATASETS.TRAIN = ("train_med_coco",)
# cfg.DATASETS.VAL = ("val_med_coco",)
# cfg.DATASETS.TEST = ("test_med_coco",)
cfg.DATALOADER.NUM_WORKERS = 4

In [ ]:
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
# cfg.SOLVER.WARMUP_ITERS = 1000
# cfg.SOLVER.MAX_ITER = 1500
# cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.WARMUP_ITERS = 10
cfg.SOLVER.MAX_ITER = 200
cfg.SOLVER.STEPS = (20, 30)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST

In [ ]:
print(cfg.dump())

In [ ]:
# dataloader = build_detection_train_loader(
#             cfg, per_gpu_bs=0, seed=0, rank=0, world_size=0, context='test'
#         )

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST=0.05

In [ ]:
import torch

In [ ]:
# torch.cuda.dev

In [ ]:


cfg.MODEL.DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu'
cfg.MODEL.DEVICE

In [ ]:
cfg.OUTPUT_DIR

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80

cfg.TEST.EVAL_PERIOD = 3

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# predictor = DefaultPredictor(cfg)

#Call the COCO Evaluator function and pass the Validation Dataset
evaluator = COCOEvaluator("test_med_coco", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "test_med_coco")

#Use the created predicted model in the previous step
inference_on_dataset(predictor.model, val_loader, evaluator)

In [ ]:

nuts_metadata = MetadataCatalog.get('test_med_coco')
dataset_dicts = DatasetCatalog.get("test_med_coco")

In [ ]:
def predict_batch(predictor, im_list):
    with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
        inputs_list = []
        for original_image in im_list:
            # Apply pre-processing to image.
            if predictor.input_format == "RGB":
                # whether the model expects BGR inputs or RGB
                original_image = original_image[:, :, ::-1]
            height, width = original_image.shape[:2]
            # Do not apply original augmentation, which is resize.
            # image = predictor.aug.get_transform(original_image).apply_image(original_image)
            image = original_image
            image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
            inputs = {"image": image, "height": height, "width": width}
            inputs_list.append(inputs)
        predictions = predictor.model(inputs_list)
        return predictions

In [ ]:

for d in random.sample(dataset_dicts, 3):
    print(d)
    img = cv2.imread(d["file_name"])
    outputs_list = predict_batch(predictor, [img])
    # print(outputs_list)
    visualizer = Visualizer(img[:, :], metadata=nuts_metadata , scale=1.2)
    out = visualizer.draw_instance_predictions(outputs_list[0]["instances"].to("cpu"))
    # vis = visualizer.draw_dataset_dict(d)
    plt.imshow(out.get_image()[:, :])
    plt.show()

In [ ]:
# evaluator = COCOEvaluator("train_med_coco", cfg, False, output_dir=cfg.OUTPUT_DIR)
# # val_loader = build_detection_test_loader(cfg, "train_med_coco")
# # inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST